<a href="https://colab.research.google.com/github/aimalz/CASTORpz/blob/main/TheLastMetric.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!ls /content/drive/MyDrive/Research/CASTORpz

ext_phot.fits  knn_colors.py  unperturbed_mags.fits


In [3]:
from astropy.table import Table
prepend = '/content/drive/MyDrive/Research/CASTORpz/'
unp = Table.read(prepend+"unperturbed_mags.fits")
print(len(unp))

698211


In [4]:
!pip install astropy pzflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 62.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.2/72.2 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.9/154.9 KB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.9/87.9 KB 11.3 MB/s eta 0:00:00
  Created wheel for jax: filename=jax-0.4.3-py3-none-any.whl size=1384905 sha256=41cf0c5ab4f9b0c6db4dafa3ca6a51d10399c17aa36e46e5bef215b6ab608cbe
  Stored in directory: /root/.cache/pip/wheels/46/38/24/69c60b74118b6de655c576685cbe9c654f868ac27597ee5c98
Successfully built jax
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.3.25+cuda11.cudnn805
    Uninstalling jaxlib-0.3.25+cuda11.cudnn805:
      Successfully uninstalled jaxlib-0.3

In [5]:
from pzflow import Flow
import jax.numpy as jnp
import pandas as pd 
import numpy as np
from astropy.table import Table
from pzflow import Flow, FlowEnsemble
from pzflow.distributions import Uniform
from pzflow.bijectors import Chain, StandardScaler, NeuralSplineCoupling, ColorTransform, InvSoftplus, RollingSplineCoupling, ShiftBounds

In [6]:
# load data
def getTrueY(test_cat, mag_col_names, y_col_name):
    test_cat = Table(test_cat, masked=True, copy=True)
    # remove nans
    for col in mag_col_names:
        test_cat[col].mask = np.isnan(test_cat[col].data) | test_cat[col].mask
        test_cat = test_cat[~test_cat[col].mask] # then remove nans from test set
            
    true_y = test_cat[y_col_name]
    return true_y.filled()

In [7]:
unp = Table.read(prepend+"unperturbed_mags.fits")
mock = Table.read(prepend+"ext_phot.fits")
mask = unp["Euclid_VIS_MAG"] <= 24.5

In [8]:
unp = Table(unp.as_array().byteswap().newbyteorder('=')).to_pandas()
mock = Table(mock.as_array().byteswap().newbyteorder('=')).to_pandas()
mask = unp["Euclid_VIS_MAG"] <= 24.5

In [9]:
mock

,ID,photoz,castor_uv_MAG,castor_uv_MAGERR,castor_u_MAG,castor_u_MAGERR,castor_g_MAG,castor_g_MAGERR,LSST_u_MAG,LSST_u_MAGERR,...,Roman_129_MAG,Roman_129_MAGERR,Roman_158_MAG,Roman_158_MAGERR,Roman_184_MAG,Roman_184_MAGERR,hlr,castor_uv_UNPMAG,castor_u_UNPMAG,castor_g_UNPMAG
0,72057594037927936,-2.543659e-75,-7.394096e-86,8.694793e-209,-2.048443e-67,3.117574e-06,1.758084e+207,1.872801e+72,9.466568e-67,-2.565944e-198,...,-9.414230e+147,5.272483e-269,-2.859142e+29,2.086167e-176,-1.569907e+52,-6.030651e-129,-1.598647e+276,1.702424e+89,-2.631514e+45,-6.597119e-23
1,144115188075855872,-9.465570e-304,4.268542e-106,1.902266e+83,-3.232440e+50,-9.064007e+32,-6.477126e+12,2.030104e+298,8.800539e-123,1.133158e+282,...,1.586146e+164,1.032409e-171,-4.894900e-46,3.203467e+30,1.049818e+148,3.002510e-258,1.559091e+62,1.618672e-102,-9.301418e+296,-6.743390e+285
2,216172782113783808,1.684657e-281,-1.041674e-07,-5.549261e+248,4.656272e+226,7.712731e+236,6.303285e+87,-3.652806e-275,2.222899e+145,-1.821135e+219,...,-3.048966e-237,-3.831640e+143,-8.370918e-298,-1.089383e+247,-3.520815e-102,2.297958e+164,-2.171952e+252,1.124811e+176,-1.085538e+170,-1.172500e-66
3,360287970189639680,-3.100461e-282,7.217290e-123,-3.031195e+289,-2.607252e-296,9.712997e-111,7.546735e-67,-5.753622e+106,-9.225312e-141,2.527182e+32,...,-6.681381e-87,-9.000401e-200,1.322323e-09,-2.047856e+177,4.413927e+107,1.998570e-98,-2.296296e-231,-1.595520e-162,2.367488e+164,3.296123e-38
4,432345564227567616,1.314750e+167,NaN,7.916218e+47,-1.684993e-85,-9.602763e-79,-3.711586e-181,-2.616893e+95,-5.284670e-240,-1.262753e+178,...,2.682030e+300,3.401996e+306,-2.730482e-63,-2.782887e+20,-2.191507e+205,2.223287e+10,1.514829e-206,2.372232e-26,-9.781708e-253,1.091602e+170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
698206,5599960054821289984,-1.388149e-10,-4.152023e-278,-3.320579e-208,-7.040234e-28,-2.816637e-130,2.222513e+30,2.478456e+04,-2.688082e-09,7.434748e+210,...,2.519193e-09,-4.050484e+121,-3.804567e-46,3.220727e-142,-9.107487e+167,-1.916811e-59,-4.256022e+250,-6.668074e+229,6.586876e-219,1.126141e+85
698207,6176420807124713472,-2.896917e-112,-4.979073e+245,-4.549348e-216,-1.890170e-66,-3.097297e+133,-2.722765e+280,-1.303209e+151,2.364686e-05,1.428619e+60,...,-2.098060e+164,9.426555e-215,-8.865919e-219,-8.529417e-72,1.653597e+130,-6.154615e-202,5.711441e+274,-7.011951e-131,-8.276250e-173,-1.298237e-02
698208,6320535995200569344,-2.736258e+120,-1.055698e-122,-5.654874e-270,-2.320765e-257,2.786414e-278,2.164390e-07,1.072619e+115,-4.795587e-279,-5.261573e-126,...,-2.170770e-46,-2.316481e-164,1.994077e-143,-2.199036e+298,5.492368e-124,-2.328237e+185,2.767767e-45,3.490711e-208,-3.930075e+113,-1.703754e+154
698209,6392593589238497280,5.116804e+48,-8.677415e+167,2.218994e-28,-5.450139e-103,-1.715517e+35,NaN,-1.697838e-79,7.998225e-86,8.040605e+242,...,-7.808172e-240,-4.097964e+251,3.558623e-295,7.115363e-112,-7.687975e-220,2.048935e+69,-2.526391e+89,1.142059e+32,-7.767144e-202,2.173469e-52


In [10]:
CASTOR_baseline = Table.from_pandas(mock[mask]) #ignoring training data outside the Euclid flux limit 

In [11]:
names_phot = ["ID", "photoz",
              "LSST_g_MAG", "LSST_g_MAGERR", 
              "LSST_r_MAG", "LSST_r_MAGERR",
              "LSST_i_MAG", "LSST_i_MAGERR", 
              "LSST_z_MAG", "LSST_z_MAGERR",
              "castor_uv_MAG", "castor_uv_MAGERR", 
              "castor_u_MAG", "castor_u_MAGERR", 
              "castor_g_MAG", "castor_g_MAGERR"]
available_os = ["baseline"]
names = [
    "baseline",
]

In [12]:
os_names = dict(zip(available_os, names))
colors = ["k"] #, "plum", "cornflowerblue", "#2ca02c", "gold", "tomato"]
os_colors = dict(zip(available_os, colors))

In [13]:
# put data in expected format for TLM

LSST_g_mag = getTrueY(test_cat=CASTOR_baseline, mag_col_names=names_phot, y_col_name="LSST_g_MAG")
LSST_r_mag = getTrueY(test_cat=CASTOR_baseline, mag_col_names=names_phot, y_col_name="LSST_r_MAG")
LSST_i_mag = getTrueY(test_cat=CASTOR_baseline, mag_col_names=names_phot, y_col_name="LSST_i_MAG")
LSST_z_mag = getTrueY(test_cat=CASTOR_baseline, mag_col_names=names_phot, y_col_name="LSST_z_MAG")

LSST_g_mag_ERR = getTrueY(test_cat=CASTOR_baseline, mag_col_names=names_phot, y_col_name="LSST_g_MAGERR")
LSST_r_mag_ERR = getTrueY(test_cat=CASTOR_baseline, mag_col_names=names_phot, y_col_name="LSST_r_MAGERR")
LSST_i_mag_ERR = getTrueY(test_cat=CASTOR_baseline, mag_col_names=names_phot, y_col_name="LSST_i_MAGERR")
LSST_z_mag_ERR = getTrueY(test_cat=CASTOR_baseline, mag_col_names=names_phot, y_col_name="LSST_z_MAGERR")

CASTOR_g_mag = getTrueY(test_cat=CASTOR_baseline, mag_col_names=names_phot, y_col_name="castor_g_MAG")
CASTOR_u_mag = getTrueY(test_cat=CASTOR_baseline, mag_col_names=names_phot, y_col_name="castor_u_MAG")
CASTOR_uv_mag = getTrueY(test_cat=CASTOR_baseline, mag_col_names=names_phot, y_col_name="castor_uv_MAG")

CASTOR_g_mag_ERR = getTrueY(test_cat=CASTOR_baseline, mag_col_names=names_phot, y_col_name="castor_g_MAGERR")
CASTOR_u_mag_ERR = getTrueY(test_cat=CASTOR_baseline, mag_col_names=names_phot, y_col_name="castor_u_MAGERR")
CASTOR_uv_mag_ERR = getTrueY(test_cat=CASTOR_baseline, mag_col_names=names_phot, y_col_name="castor_uv_MAGERR")

ID = getTrueY(test_cat=CASTOR_baseline, mag_col_names=names_phot, y_col_name="ID")
z_true = getTrueY(test_cat=CASTOR_baseline, mag_col_names=names_phot, y_col_name="photoz")



In [15]:
#df = pd.DataFrame({'CASTOR_ID': data_array_sorted_CASTOR_wID[:,0] , 'error': data_array_sorted_CASTOR_wID[:,1], 
#                   'g-r': data_array_sorted_CASTOR_wID[:,2] - data_array_sorted_CASTOR_wID[:,3], 
#                   'r-i': data_array_sorted_CASTOR_wID[:,3] - data_array_sorted_CASTOR_wID[:,4], 
#                   'i-z': data_array_sorted_CASTOR_wID[:,4] - data_array_sorted_CASTOR_wID[:,5], 
#                   'true_z': data_array_sorted_CASTOR_wID[:,6]})

catalogs = dict()
for os in available_os:
    
#     cat = pd.DataFrame({'CASTOR_ID': ID, 'z_true': z_true, 
#                              'LSST_g_mag': LSST_g_mag, 'LSST_g_mag_ERR': LSST_g_mag_ERR, 
#                              'LSST_r_mag': LSST_r_mag, 'LSST_r_mag_ERR': LSST_r_mag_ERR, 
#                              'LSST_i_mag': LSST_i_mag, 'LSST_i_mag_ERR': LSST_i_mag_ERR, 
#                              'LSST_z_mag': LSST_z_mag, 'LSST_z_mag_ERR': LSST_z_mag_ERR, 
#                              'CASTOR_uv_mag': LSST_uv_mag, 'LSST_uv_mag_ERR': LSST_uv_mag_ERR, 
#                              'CASTOR_u_mag': LSST_u_mag, 'LSST_u_mag_ERR': LSST_u_mag_ERR, 
#               'CASTOR_g_mag': LSST_g_mag, 'LSST_g_mag_ERR': LSST_g_mag_ERR})
    
    # this will need to change to accomodate multiple catalogs, e.g. LSST only or LSST + CASTOR
    
    cat = pd.DataFrame({'CASTOR_ID': ID, 'z_true': z_true, 
                             'r': LSST_r_mag,
                             'g-r': LSST_g_mag - LSST_r_mag, 
                             'r-i': LSST_r_mag - LSST_i_mag, 
                             'i-z': LSST_i_mag - LSST_z_mag, 
                             'uv-u': CASTOR_uv_mag - CASTOR_u_mag, 
                             'u-g': CASTOR_u_mag - CASTOR_g_mag
                      }).set_index('CASTOR_ID')
    catalogs[os] = cat.dropna()
    means = catalogs[os].mean(axis=0).values
    stds = catalogs[os].std(axis=0).values

/usr/local/lib/python3.8/dist-packages/numpy/core/_methods.py:48: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
/usr/local/lib/python3.8/dist-packages/pandas/core/nanops.py:933: RuntimeWarning: overflow encountered in square
  sqr = _ensure_numeric((avg - values) ** 2)


In [16]:
cat

,z_true,r,g-r,r-i,i-z,uv-u,u-g
CASTOR_ID,,,,,,,
216172782113783808,1.684657e-281,1.027149e-67,-1.292871e-28,1.027149e-67,3.085231e+166,-4.656272e+226,4.656272e+226
360287970189639680,-3.100461e-282,6.469310e+186,-6.469310e+186,6.469310e+186,-2.308197e+33,7.217290e-123,-7.546735e-67
1224979098644774912,3.070734e+90,-1.828198e-44,-3.201130e+261,7.839844e+110,9.390153e+145,-6.857277e+279,1.532873e+50
1585267068834414592,-1.423971e-104,2.341162e-161,1.059528e+89,2.341162e-161,-7.471119e-239,-7.750651e+206,7.750651e+206
1945555039024054272,-1.442478e-81,-7.720035e-105,7.720035e-105,3.470133e+106,-7.082745e+185,1.268246e+243,-1.134215e+148
...,...,...,...,...,...,...,...
5167614490593722368,-5.525235e+18,-5.860362e+51,5.860362e+51,-5.860362e+51,8.153834e-104,7.191646e+186,-7.194623e+186
5383787272707506176,1.783241e+279,-4.994289e-260,3.797509e-181,-5.016184e+147,5.016184e+147,-5.157640e-141,3.488979e-66
5455844866745434112,1.401974e-306,1.224495e-122,-1.224495e-122,1.224495e-122,7.752034e-125,-3.279503e+281,4.247980e+106


In [20]:


mins = [0, 15, -1, -1, -1, -1, -1]
maxs = [3, 35, 5, 5, 5, 5, 5]

# first I create a bijector chain
# the first bijection is a standard scaler - but I'm not actually using it for standard scaling
#     I set the mean and std so that it maps the redshift range (0, 3.2) onto (-5, 5), which is 
#     the domain of the NeuralSplineCoupling
# the second bijection is a NeuralSplineCoupling. I told it to expect 6 conditions,
#     which will be the r mag and the galaxy colors
# bijector = Chain(
#     StandardScaler(np.atleast_1d(1.6), np.atleast_1d(0.32)),
#     NeuralSplineCoupling(n_conditions=6)
# )
bijector = Chain(
    # InvSoftplus(z_col, sharpness),
    # StandardScaler(means, stds),
    ShiftBounds(mins, maxs, B=5),
    RollingSplineCoupling(nlayers=7, K=16, n_conditions=6),
)

# I set the latent distribution to a Uniform over (-5, 5)
# this range was chosen to match the NeuralSplineCoupling domain
# I chose a Uniform since all of the redshifts are drawn from a compact domain
# latent = Uniform(1, 5) # did the syntax here change?
latent = Uniform(1,5)#(-5, 5), (-5, 5), (-5, 5), (-5, 5), (-5, 5), (-5, 5), (-5, 5))

In [21]:
# create a dictionary that will hold all the ensembles
ensembles = dict()

# create the baseline flows
for os in available_os:
    # the data column is the one that is sampled and transformed by the flow
    data_columns = ["z_true"]
    # the conditional columns are the columns that the flow is conditioned on
    conditional_columns = ["r", "u-g", "g-r", "r-i", "i-z", "uv-u"] # different colors than LSST  
    # save some info with the flow
    info = f"Models z_true conditioned on galaxy colors and r mag from os {os}. K=16"
    
    # instantiate and save the flow
    flowEns = FlowEnsemble(data_columns = data_columns, 
                           conditional_columns = conditional_columns,
                           bijector = bijector,
                           latent = latent,
                           info = info),
                          #  N = 10)
    ensembles[os] = flowEns

In [ ]:
for os, ens in ensembles.items():
    
    # get the data and make a train and test set
    cat = catalogs[os]
    cat_train = cat.sample(frac = 0.8)
    cat_test = cat.drop(cat_train.index)
    
    # train the flow on the given learning rate schedule
    loss = ens[0].train(cat_train[data_columns+conditional_columns], convolve_errs=False,
                       epochs = 30, seed = 123)
    # loss2 = ens.train(cat_train, convolve_errs=True,
    #                    epochs = 30, seed = 312)
    # loss3 = ens.train(cat_train, convolve_errs=True,
    #                    epochs = 30, seed = 231)
    
    # losses = {fname : # for each flow trained in the ensemble...
    #               [float(loss) # save the list of training losses
    #                for lossDict in [loss1]#, loss2, loss3]
    #                for loss in lossDict[fname]]
    #           for fname in loss1}
    
    # print the train and test loss
    train_loss = -np.mean(ens.log_prob(cat_train))
    test_loss = -np.mean(ens.log_prob(cat_test))
    print(os, train_loss, test_loss)
    
    # save the ensemble
    ens.save(f"trained_flows/pzflow_ensemble_for_{os}.pkl")
    # and the losses
    with open(f"trained_flows/losses_for_{os}.pkl", "wb") as file:
        pickle.dump({"losses": losses, 
                     "train loss": train_loss, 
                     "test loss": test_loss},
                    file)

In [ ]:
loss

And now an error because it's looking at flows trained on catalogs with different keyword names?

dependencies: pzflow, pandas, jax

In [ ]:
from pzflow import Flow
import jax.numpy as jnp
import pandas as pd 

import matplotlib.pyplot as plt

In [ ]:
from astropy.table import Table

## CASTOR COSMOS synthetic catalogs

In [ ]:
# load data

unp = Table.read("unperturbed_mags.fits")
mock = Table.read("ext_phot.fits")
mask = unp["Euclid_VIS_MAG"] <= 24.5

CASTOR_baseline = mock[mask] #ignoring training data outside the Euclid flux limit 

names_phot = ["ID", "photoz",
              "LSST_g_MAG", "LSST_g_MAGERR", 
              "LSST_r_MAG", "LSST_r_MAGERR",
              "LSST_i_MAG", "LSST_i_MAGERR", 
              "LSST_z_MAG", "LSST_z_MAGERR"
              "castor_uv_MAG", "castor_uv_MAGERR", 
              "castor_u_MAG", "castor_u_MAGERR", 
              "castor_g_MAG", "castor_g_MAGERR"]

available_os = ["baseline"]
names = [
    "baseline",
]

os_names = dict(zip(available_os, names))
colors = ["k"] #, "plum", "cornflowerblue", "#2ca02c", "gold", "tomato"]
os_colors = dict(zip(available_os, colors))

# put data in expected format for TLM 

LSST_g_mag = getTrueY(test_cat=CASTOR_baseline, mag_col_names=names_phot, y_col_name="LSST_g_MAG")
LSST_r_mag = getTrueY(test_cat=CASTOR_baseline, mag_col_names=names_phot, y_col_name="LSST_r_MAG")
LSST_i_mag = getTrueY(test_cat=CASTOR_baseline, mag_col_names=names_phot, y_col_name="LSST_i_MAG")
LSST_z_mag = getTrueY(test_cat=CASTOR_baseline, mag_col_names=names_phot, y_col_name="LSST_z_MAG")

LSST_g_mag_ERR = getTrueY(test_cat=CASTOR_baseline, mag_col_names=names_phot, y_col_name="LSST_g_MAGERR")
LSST_r_mag_ERR = getTrueY(test_cat=CASTOR_baseline, mag_col_names=names_phot, y_col_name="LSST_r_MAGERR")
LSST_i_mag_ERR = getTrueY(test_cat=CASTOR_baseline, mag_col_names=names_phot, y_col_name="LSST_i_MAGERR")
LSST_z_mag_ERR = getTrueY(test_cat=CASTOR_baseline, mag_col_names=names_phot, y_col_name="LSST_z_MAGERR")

CASTOR_g_mag = getTrueY(test_cat=CASTOR_baseline, mag_col_names=names_phot, y_col_name="CASTOR_g_MAG")
CASTOR_r_mag = getTrueY(test_cat=CASTOR_baseline, mag_col_names=names_phot, y_col_name="CASTOR_r_MAG")
CASTOR_i_mag = getTrueY(test_cat=CASTOR_baseline, mag_col_names=names_phot, y_col_name="CASTOR_i_MAG")
CASTOR_z_mag = getTrueY(test_cat=CASTOR_baseline, mag_col_names=names_phot, y_col_name="CASTOR_z_MAG")

CASTOR_g_mag_ERR = getTrueY(test_cat=CASTOR_baseline, mag_col_names=names_phot, y_col_name="CASTOR_g_MAGERR")
CASTOR_r_mag_ERR = getTrueY(test_cat=CASTOR_baseline, mag_col_names=names_phot, y_col_name="CASTOR_r_MAGERR")
CASTOR_i_mag_ERR = getTrueY(test_cat=CASTOR_baseline, mag_col_names=names_phot, y_col_name="CASTOR_i_MAGERR")
CASTOR_z_mag_ERR = getTrueY(test_cat=CASTOR_baseline, mag_col_names=names_phot, y_col_name="CASTOR_z_MAGERR")

ID = getTrueY(test_cat=CASTOR_baseline, mag_col_names=names_phot, y_col_name="ID")
z_true = getTrueY(test_cat=CASTOR_baseline, mag_col_names=names_phot, y_col_name="photoz")

df = pd.DataFrame({'CASTOR_ID': data_array_sorted_CASTOR_wID[:,0] , 'error': data_array_sorted_CASTOR_wID[:,1], 
                   'g-r': data_array_sorted_CASTOR_wID[:,2] - data_array_sorted_CASTOR_wID[:,3], 
                   'r-i': data_array_sorted_CASTOR_wID[:,3] - data_array_sorted_CASTOR_wID[:,4], 
                   'i-z': data_array_sorted_CASTOR_wID[:,4] - data_array_sorted_CASTOR_wID[:,5], 
                   'true_z': data_array_sorted_CASTOR_wID[:,6]})

In [ ]:
# load the catalogs
catalogs = dict()
for os in available_os:
    
#     cat = pd.DataFrame({'CASTOR_ID': ID, 'z_true': z_true, 
#                              'LSST_g_mag': LSST_g_mag, 'LSST_g_mag_ERR': LSST_g_mag_ERR, 
#                              'LSST_r_mag': LSST_r_mag, 'LSST_r_mag_ERR': LSST_r_mag_ERR, 
#                              'LSST_i_mag': LSST_i_mag, 'LSST_i_mag_ERR': LSST_i_mag_ERR, 
#                              'LSST_z_mag': LSST_z_mag, 'LSST_z_mag_ERR': LSST_z_mag_ERR, 
#                              'CASTOR_uv_mag': LSST_uv_mag, 'LSST_uv_mag_ERR': LSST_uv_mag_ERR, 
#                              'CASTOR_u_mag': LSST_u_mag, 'LSST_u_mag_ERR': LSST_u_mag_ERR, 
#               'CASTOR_g_mag': LSST_g_mag, 'LSST_g_mag_ERR': LSST_g_mag_ERR})
    
    # this will need to change to accomodate multiple catalogs, e.g. LSST only or LSST + CASTOR
    
    cat = pd.DataFrame({'CASTOR_ID': ID, 'z_true': z_true, 
                             'r': LSST_r_mag,
                             'g-r': LSST_g_mag - LSST_r_mag, 
                             'r-i': LSST_r_mag - LSST_i_mag, 
                             'i-z': LSST_i_mag - LSST_z_mag, 
                             'uv-u': CASTOR_uv_mag - CASTOR_u_mag; 
                             'u-g': CASTOR_u_mag - CASTOR_g_mag
                      })
    
    catalogs[os] = cat.dropna()

# Instantiate and Train 

### Does this run 'as is'?

In [ ]:
# first I create a bijector chain
# the first bijection is a standard scaler - but I'm not actually using it for standard scaling
#     I set the mean and std so that it maps the redshift range (0, 3.2) onto (-5, 5), which is 
#     the domain of the NeuralSplineCoupling
# the second bijection is a NeuralSplineCoupling. I told it to expect 6 conditions,
#     which will be the r mag and the galaxy colors
bijector = Chain(
    StandardScaler(np.atleast_1d(1.6), np.atleast_1d(0.32)),
    NeuralSplineCoupling(n_conditions=6)
)

# I set the latent distribution to a Uniform over (-5, 5)
# this range was chosen to match the NeuralSplineCoupling domain
# I chose a Uniform since all of the redshifts are drawn from a compact domain
latent = Uniform(1)

# create a dictionary that will hold all the ensembles
ensembles = dict()

# create the baseline flows
for os in available_os:

    # the data column is the one that is sampled and transformed by the flow
    data_columns = ["z_true"]
    # the conditional columns are the columns that the flow is conditioned on
    conditional_columns = ["r", "u-g", "g-r", "r-i", "i-z", "uv-u"] # different colors than LSST  

    # save some info with the flow
    info = f"Models z_true conditioned on galaxy colors and r mag from os {os}. K=16"

    # instantiate and save the flow
    flowEns = FlowEnsemble(data_columns = data_columns, 
                           conditional_columns = conditional_columns,
                           bijector = bijector,
                           latent = latent,
                           info = info,
                           N = 10)

    ensembles[os] = flowEns
    

for os, ens in ensembles.items():
    
    # get the data and make a train and test set
    cat = catalogs[os]
    cat_train = cat.sample(frac = 0.8)
    cat_test = cat.drop(cat_train.index)
    
    # train the flow on the given learning rate schedule
    loss1 = ens.train(cat_train, sample_errs=True,
                       optimizer = adam(step_size = 1e-3),
                       epochs = 100, seed = 123)
    loss2 = ens.train(cat_train, sample_errs=True,
                       optimizer = adam(step_size = 2e-4),
                       epochs = 100, seed = 312)
    loss3 = ens.train(cat_train, sample_errs=True,
                       optimizer = adam(step_size = 1e-4),
                       epochs = 50, seed = 231)
    
    losses = {fname : # for each flow trained in the ensemble...
                  [float(loss) # save the list of training losses
                   for lossDict in [loss1, loss2, loss3]
                   for loss in lossDict[fname]]
              for fname in loss1}
    
    # print the train and test loss
    train_loss = -np.mean(ens.log_prob(cat_train))
    test_loss = -np.mean(ens.log_prob(cat_test))
    print(os, train_loss, test_loss)
    
    # save the ensemble
    ens.save(f"trained_flows/pzflow_ensemble_for_{os}.pkl")
    # and the losses
    with open(f"trained_flows/losses_for_{os}.pkl", "wb") as file:
        pickle.dump({"losses": losses, 
                     "train loss": train_loss, 
                     "test loss": test_loss},
                    file)

NameError: ignored

# from github/aimalz/TheLastMetric/blob/master/training_flows_July23.ipynb

pull the data just to get formatting

In [ ]:
# !wget https://storage.googleapis.com/ahw2019/for_malz_and_lanusse.tar.gz
# !tar -xzf for_malz_and_lanusse.tar.gz
# !mv for_malz_and_lanusse dataset



more dataset/readme.txt



# list of available catalogs
available_os = ["run_1_4_y10", "run_4_38_y10", "run_10_92_y10", "run_4_34_y10", "run_7_61_y10", "run_9_86_y10"]
names = [
    "baseline_v1_5_10yrs",
    "footprint_stuck_rollingv1_5_10yrs",
    "ddf_heavy_nexp2_v1_6_10yrs",
    "footprint_newAv1_5_10yrs",
    "third_obs_pt60v1_5_10yrs",
    "barebones_v1_6_10yrs",
]
os_names = dict(zip(available_os, names))
colors = ["k", "plum", "cornflowerblue", "#2ca02c", "gold", "tomato"]
os_colors = dict(zip(available_os, colors))


# column names of the catalogs
names_z=('ID', 'z_true', 'z_phot', 'dz_phot', 'NN', 'N_train')
names_phot=(
    'ID', 'z_true', 
    'u', 'u_err',
    'g', 'g_err',
    'r', 'r_err',
    'i', 'i_err',
    'z', 'z_err',
    'y', 'y_err',
    'u-g', 'u-g_err',
    'g-r', 'g-r_err',
    'r-i', 'r-i_err',
    'i-z', 'i-z_err',
    'z-y', 'z-y_err',
)

# load the catalogs
catalogs = dict()
for os in available_os:
    z_cat = pd.read_csv(f"dataset/{os}/zphot.cat", names=names_z, delim_whitespace=True, skiprows=1)
    phot_cat = pd.read_csv(f"dataset/{os}/test.cat", names=names_phot, delim_whitespace=True)
    cat = z_cat.merge(phot_cat)
    catalogs[os] = cat.dropna()

instantiate the model of the photometric space, train it on the catalogs for LSST with and without CASTOR, and save it (two versions rather than 6 OSs)

In [ ]:
# first I create a bijector chain
# the first bijection is a standard scaler - but I'm not actually using it for standard scaling
#     I set the mean and std so that it maps the redshift range (0, 3.2) onto (-5, 5), which is 
#     the domain of the NeuralSplineCoupling
# the second bijection is a NeuralSplineCoupling. I told it to expect 6 conditions,
#     which will be the r mag and the galaxy colors
bijector = Chain(
    StandardScaler(np.atleast_1d(1.6), np.atleast_1d(0.32)),
    NeuralSplineCoupling(n_conditions=6)
)

# I set the latent distribution to a Uniform over (-5, 5)
# this range was chosen to match the NeuralSplineCoupling domain
# I chose a Uniform since all of the redshifts are drawn from a compact domain
latent = Uniform((-5, 5))

# create a dictionary that will hold all the ensembles
ensembles = dict()

# create the baseline flows
for os in available_os:

    # the data column is the one that is sampled and transformed by the flow
    data_columns = ["z_true"]
    # the conditional columns are the columns that the flow is conditioned on
    conditional_columns = ["r", "u-g", "g-r", "r-i", "i-z", "z-y"]

    # save some info with the flow
    info = f"Models z_true conditioned on galaxy colors and r mag from os {os}. K=16"

    # instantiate and save the flow
    flowEns = FlowEnsemble(data_columns = data_columns, 
                           conditional_columns = conditional_columns,
                           bijector = bijector,
                           latent = latent,
                           info = info,
                           N = 10)

    ensembles[os] = flowEns
    
    
    
    
%%time

for os, ens in ensembles.items():
    
    # get the data and make a train and test set
    cat = catalogs[os]
    cat_train = cat.sample(frac = 0.8)
    cat_test = cat.drop(cat_train.index)
    
    # train the flow on the given learning rate schedule
    loss1 = ens.train(cat_train, sample_errs=True,
                       optimizer = adam(step_size = 1e-3),
                       epochs = 100, seed = 123)
    loss2 = ens.train(cat_train, sample_errs=True,
                       optimizer = adam(step_size = 2e-4),
                       epochs = 100, seed = 312)
    loss3 = ens.train(cat_train, sample_errs=True,
                       optimizer = adam(step_size = 1e-4),
                       epochs = 50, seed = 231)
    
    losses = {fname : # for each flow trained in the ensemble...
                  [float(loss) # save the list of training losses
                   for lossDict in [loss1, loss2, loss3]
                   for loss in lossDict[fname]]
              for fname in loss1}
    
    # print the train and test loss
    train_loss = -np.mean(ens.log_prob(cat_train))
    test_loss = -np.mean(ens.log_prob(cat_test))
    print(os, train_loss, test_loss)
    
    # save the ensemble
    ens.save(f"trained_flows/pzflow_ensemble_for_{os}.pkl")
    # and the losses
    with open(f"trained_flows/losses_for_{os}.pkl", "wb") as file:
        pickle.dump({"losses": losses, 
                     "train loss": train_loss, 
                     "test loss": test_loss},
                    file)

# from github/aimalz/TheLastMetric/blob/master/MAFVariationalMutualInformationPzFlow.ipynb

open the models of redshift-photometry space and the catalogs

In [ ]:
flows = {}
for os in available_os:
  flows[os] = Flow(file=f"trained_flows/flow_for_run_{os}.pkl")
# TODO: need to experiment with different fit parameters because this might be too smooth, also does it account for photometric errors?
# TODO: check that draws from flow look like original data


# load the catalogs
catalogs = dict()
for os in available_os:
    z_cat = pd.read_csv(f"dataset/run_{os}/zphot.cat", names=names_z, delim_whitespace=True, skiprows=1)
    phot_cat = pd.read_csv(f"dataset/run_{os}/test.cat", names=names_phot, delim_whitespace=True)
    cat = z_cat.merge(phot_cat)
    catalogs[os] = cat.dropna()
    
    

evaluate the posteriors for each galaxy (totally forgot we had to do this)

In [ ]:
# # this just makes the posteriors for plotting, not sure why it uses so much memory. . .
# tx = np.linspace(0,3.5,100)
# all_logp = {}
# for which_os in available_os:
#   flow = flows[which_os]
#   cat = catalogs[which_os]
#   logp = flow.posterior(flow.info["condition_scaler"](cat), column="z_true", grid=tx)
#   all_logp[which_os] = logp
    
    

In [ ]:
all_milb = {}
for which_os in available_os:
  phot_cat = catalogs[which_os]

  mutual_information_lower_bound = flows[which_os].log_prob(flows[which_os].info["condition_scaler"](phot_cat))
  all_milb[which_os] = mutual_information_lower_bound
  print((os_names[which_os], np.sum(mutual_information_lower_bound)))
# TODO: make this an actual expected value rather than just sum
# also, shouldn't it be sum of exponential of metric value, since it should never penalize a negative value?

KEY PLOT: distribution of metric values for same galaxies with and without CASTOR

In [ ]:
# surprisingly not so different from one another
for which_os in available_os:
  mutual_information_lower_bound = all_milb[which_os].flatten()
  print((np.mean(mutual_information_lower_bound), np.std(mutual_information_lower_bound)))
  hist(mutual_information_lower_bound, bins=np.linspace(-16, 5, 100), alpha=0.5, histtype='step', 
       color=os_colors[which_os], label=os_names[which_os], density=False)
  xlabel(r'$\mathbb{E}_{z, x_{phot}} \left[ q_\theta(z | x_{phot}) \right]$')
# xlim(-10., 5.)
legend(loc='upper left')
# semilogy()

KEY PLOT: look at per-galaxy metric values as a function of redshift, then photometry

In [ ]:
fig, axs = plt.subplots(len(available_os), 1, figsize=(5, 5*len(available_os)))
for i, which_os in enumerate(available_os):
  axs[i].hist2d(z_cats[which_os]['z_true'], all_milb[which_os].flatten(), bins=[np.linspace(0., 3., 50), np.linspace(-5., 5., 100)])
  axs[i].set_xlabel('redshift')
  axs[i].set_ylabel(r'$\mathbb{E}_{z, x_{phot}} \left[ q_\theta(z | x_{phot}) \right]$')
  axs[i].set_title(os_names[which_os])
# they're different, but not visibly so
# TODO: plot violins of metric as a function of binned redshift so they're all on one set of axes? or quantiles because outlers? or box/whisker https://matplotlib.org/stable/gallery/pyplots/boxplot_demo_pyplot.html?
# TODO: normalize within redshift bins to get these on one set of axes?
# TODO sort of want sum of metric values in redshift bins, no?

take expected value of the approximated posteriors, which is the mutual information lower bound

In [ ]:
# something isn't right about the autocalculation of moments so doing it by hand
def calc_moment(vals, k):
  n = len(vals)
  outval = np.sum(vals**k) / float(n)
  return float(outval)

which_moments = range(0, 5)
moment_res = {}
for which_os in available_os:
  # print((np.mean(all_milb[which_os]), np.std(all_milb[which_os])))
  moment_res[which_os] = []
  for i in which_moments:
    moment_res[which_os].append(calc_moment(all_milb[which_os], k=i))#sps.mstats.moment(all_milb[which_os], moment=which_moments[i], axis=0))
# print(moment_res)

# from github/aimalz/TheLastMetric/blob/master/FigureTLMvsRedshift.ipynb

KEY PLOT: distribution of TLM values with and without CASTOR including model uncertainty

In [ ]:
%pylab inline
from utils import load_data, compute_last_metric
import corner
from pzflow import Flow


# Loading data
z_cats, phot_cats, available_os, os_names, os_colors = load_data()


# Loading pre-trained flows
flows = {}
for os in available_os:
  flows[os] = [Flow(file=f"trained_flows/flow_for_run_{os}_%d.pkl"%(i+1) ) for i in range(10)]


# Computing metric for each observing strategy
all_tlm = {}
for which_os in available_os:
  all_tlm[which_os] = np.stack([(compute_last_metric(f,
                                          phot_cats[which_os],
                                          z_cats[which_os], entropy_nbins=60)) for f in flows[which_os] ], axis=0)
  print((os_names[which_os], np.mean(all_tlm[which_os]), np.std(np.mean(all_tlm[which_os], axis=1))))
    
    
    

figure(figsize=(7,5))
for which_os in available_os:
  hist(np.mean(all_tlm[which_os],axis=1), 32, range=[2.95, 3.35], alpha=0.6,
       color=os_colors[which_os], label=os_names[which_os])
  axvline(np.mean(all_tlm[which_os]), color=os_colors[which_os], linewidth=2)
legend(ncol=2)
xlabel(chr(0x05ea))
savefig('metrics.pdf', bbox_inches = 'tight', pad_inches = 0 )